In [7]:
from games_setup import *
from SBMLLint.games.som import SOM
from SBMLLint.games.mesgraph import MESGraph

In [30]:
simple = load_file_from_curated_data(10)

In [31]:
simple.reactions

[J0: MKKK -> MKKK_P; uVol * V1 * MKKK / ((1 + pow(MAPK_PP / Ki, n)) * (K1 + MKKK)),
 J1: MKKK_P -> MKKK; uVol * V2 * MKKK_P / (KK2 + MKKK_P),
 J2: MKK -> MKK_P; uVol * k3 * MKKK_P * MKK / (KK3 + MKK),
 J3: MKK_P -> MKK_PP; uVol * k4 * MKKK_P * MKK_P / (KK4 + MKK_P),
 J4: MKK_PP -> MKK_P; uVol * V5 * MKK_PP / (KK5 + MKK_PP),
 J5: MKK_P -> MKK; uVol * V6 * MKK_P / (KK6 + MKK_P),
 J6: MAPK -> MAPK_P; uVol * k7 * MKK_PP * MAPK / (KK7 + MAPK),
 J7: MAPK_P -> MAPK_PP; uVol * k8 * MKK_PP * MAPK_P / (KK8 + MAPK_P),
 J8: MAPK_PP -> MAPK_P; uVol * V9 * MAPK_PP / (KK9 + MAPK_PP),
 J9: MAPK_P -> MAPK; uVol * V10 * MAPK_P / (KK10 + MAPK_P)]

In [32]:
m = MESGraph(simple)

In [33]:
m.nodes

NodeView(({MKKK}, {MKKK_P}, {MKK}, {MKK_P}, {MKK_PP}, {MAPK}, {MAPK_P}, {MAPK_PP}))

In [34]:
m.edges

OutEdgeView([])

In [20]:
INITIAL_NODES = 14
INITIAL_EDGES = 0
FINAL_NODES = 6
FINAL_EDGES = 6
UNIUNI0 = 0
UNIUNI1 = 7
UNIUNI2 = 8
UNIMULTI = 2
MULTIUNI = 13
INEQUAL1 = 14
INEQUAL2 = 15
AA = "AA"
CN = "Cn"
DFG = "DFG"
E1 = "E1"
E2 = "E2"
FRU = "Fru"
GLY = "Gly"
MEL = "Mel"
MG = "MG"

In [48]:
simple = load_file_from_curated_data(50)
m = MESGraph(simple)

In [49]:
molecules = [som.molecules for som in m.nodes]
{simple.getMolecule('MKKK')} in molecules

False

In [50]:
m.identifier

'{DFG};{E1};{E2};{Gly};{Cn};{_3DG};{FA};{_1DG};{AA};{Man};{Glu};{Mel};{MG};{Fru}'

In [53]:
# test constructor:
print(len(m.nodes) == INITIAL_NODES)
print(len(m.edges) == INITIAL_EDGES)
dfg = simple.getMolecule(DFG)
# molecules is a list of one-molecule sets
molecules = [som.molecules for som in m.nodes]
print({simple.getMolecule(DFG)} in molecules)

True
True
True


In [59]:
# testInitializeSOMs
for node in m.nodes:
  print(type(node) == SOM)

True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [56]:
# testMakeId
identifier = ""
for key, som in enumerate(m.nodes):
  identifier = identifier + som.identifier
  if key < len(m.nodes)-1:
    identifier = identifier + ";"
print(identifier == m.identifier)

True


In [67]:
# testGetNode
aa = simple.getMolecule(AA)
aa_som = m.getNode(aa)
print(type(aa_som) == SOM)
print(aa_som.molecules == {aa})

True
True


In [78]:
# testUniUniReaction
uniuni_reaction = simple.reactions[UNIUNI0]
m.processUniUniReaction(uniuni_reaction)
dfg = m.getNode(simple.getMolecule(DFG))
e1 = m.getNode(simple.getMolecule(E1))
print(m.has_node(dfg) == True)
print(m.has_node(e1) == True)
print(dfg == e1)

True
True
True


In [79]:
# testUniMultiReaction
unimulti_reaction = simple.reactions[UNIMULTI]
print(unimulti)
m.processUniMultiReaction(unimulti_reaction)
prods = [m.getNode(product.molecule) for product in unimulti_reaction.products]
dfg = m.getNode(simple.getMolecule(DFG))
for prod in prods:
  print(m.has_edge(prod, dfg))

v3: DFG -> Gly + Cn; k3 * DFG
True
True


In [80]:
m.edges

OutEdgeView([({Gly}, {DFG, E1}), ({Cn}, {DFG, E1})])

In [85]:
# testMultiUniReaction
multiuni_reaction = simple.reactions[MULTIUNI]
print(multiuni_reaction)
m.processMultiUniReaction(multiuni_reaction)
reacts = [m.getNode(reactant.molecule) for reactant in multiuni_reaction.reactants]
mel = m.getNode(simple.getMolecule(MEL))
for react in reacts:
  print(m.has_edge(react, mel))

v14: Gly + Cn -> Mel; k14 * Cn * Gly
True
True


In [86]:
m.edges

OutEdgeView([({Gly}, {DFG, E1}), ({Gly}, {Mel}), ({Cn}, {DFG, E1}), ({Cn}, {Mel})])

In [93]:
# testAddArc
REACTION = "reaction"
source = [simple.getMolecule(FRU), simple.getMolecule(GLY)]
destination = [simple.getMolecule(E2)]
dummy_reaction = simple.reactions[INEQUAL2]
m.addArc(source, destination, dummy_reaction)
arc1 = [m.getNode(source[0]), 
        m.getNode(destination[0])]
arc2 = [m.getNode(source[1]), 
        m.getNode(destination[0])]
print(m.has_edge(arc1[0], arc1[1]))
print(m.has_edge(arc2[0], arc2[1]))
reaction_label1 = m.get_edge_data(arc1[0], arc1[1])[REACTION][0]
reaction_label2 = m.get_edge_data(arc2[0], arc1[1])[REACTION][0]
print(reaction_label1 == dummy_reaction.label)
print(reaction_label1 == reaction_label2)
print(m.edges)

True
True
True
True
[({Gly}, {DFG, E1}), ({Gly}, {Mel}), ({Gly}, {E2}), ({Cn}, {DFG, E1}), ({Cn}, {Mel}), ({Fru}, {E2})]


In [97]:
# test checkTypeOneError
uniuni_reaction1 = simple.reactions[7]
uniuni_reaction2 = simple.reactions[8]
inequality_reaction1 = simple.reactions[14]
inequality_reaction2 = simple.reactions[15]
m.processUniUniReaction(uniuni_reaction1)
m.processUniUniReaction(uniuni_reaction2)
aa = simple.getMolecule(AA)
cn = simple.getMolecule(CN)
mg = simple.getMolecule(MG)
print(m.checkTypeOneError((aa, cn), inequality_reaction1) == True)
print(m.checkTypeOneError((mg, aa), inequality_reaction2) == False)

We have a Type I Error...
AA  and  Cn  have the same weight by
v9: _1DG -> AA; k9 * _1DG
v8: _1DG -> Cn; k8 * _1DG

However, reaction " v15: Cn -> AA + FA + MG; k15 * Cn " implies  AA < Cn
We cannot add the arc:  AA -> Cn

True
True


In [59]:
# test analyze
k = m.analyze(Reaction.reactions)
print(k.identifier)
print(len(k.nodes) == 6)
print(len(k.edges) == 6)

We have Type I Error...
AA  and  Cn  have the same weight by
v8: _1DG -> Cn; k8 * _1DG
v9: _1DG -> AA; k9 * _1DG
v6: _3DG -> FA; k6 * _3DG
v13: Glu -> _3DG; k13 * Glu
v5: _3DG -> Cn; k5 * _3DG
v12: Man -> Glu; k12 * Man

However, reaction " v15: Cn -> AA + FA + MG; k15 * Cn " implies  AA  <  Cn

We have Type I Error...
FA  and  Cn  have the same weight by
v8: _1DG -> Cn; k8 * _1DG
v9: _1DG -> AA; k9 * _1DG
v6: _3DG -> FA; k6 * _3DG
v13: Glu -> _3DG; k13 * Glu
v5: _3DG -> Cn; k5 * _3DG
v12: Man -> Glu; k12 * Man

However, reaction " v15: Cn -> AA + FA + MG; k15 * Cn " implies  FA  <  Cn

{Gly}->{Mel}
{Gly}->{DFG, E1, E2}
{MG}->{AA, Cn, FA, Glu, Man, _1DG, _3DG}
{Fru}->{DFG, E1, E2}
{AA, Cn, FA, Glu, Man, _1DG, _3DG}->{DFG, E1, E2}
{AA, Cn, FA, Glu, Man, _1DG, _3DG}->{Mel}

True
True


In [37]:
m.analyze(Reaction.reactions)

We have Type I Error...
AA  and  Cn  have the same weight by
v8: _1DG -> Cn; k8 * _1DG
v9: _1DG -> AA; k9 * _1DG
v6: _3DG -> FA; k6 * _3DG
v13: Glu -> _3DG; k13 * Glu
v5: _3DG -> Cn; k5 * _3DG
v12: Man -> Glu; k12 * Man

However, reaction " v15: Cn -> AA + FA + MG; k15 * Cn " implies  AA  <  Cn

We have Type I Error...
FA  and  Cn  have the same weight by
v8: _1DG -> Cn; k8 * _1DG
v9: _1DG -> AA; k9 * _1DG
v6: _3DG -> FA; k6 * _3DG
v13: Glu -> _3DG; k13 * Glu
v5: _3DG -> Cn; k5 * _3DG
v12: Man -> Glu; k12 * Man

However, reaction " v15: Cn -> AA + FA + MG; k15 * Cn " implies  FA  <  Cn



{Gly}->{Mel}
{Gly}->{DFG, E1, E2}
{MG}->{AA, Cn, FA, Glu, Man, _1DG, _3DG}
{Fru}->{DFG, E1, E2}
{AA, Cn, FA, Glu, Man, _1DG, _3DG}->{DFG, E1, E2}
{AA, Cn, FA, Glu, Man, _1DG, _3DG}->{Mel}

In [63]:
dummy_reaction

v16: E2 -> Gly + Fru; k16 * E2

In [65]:
inequality_reaction2

v16: E2 -> Gly + Fru; k16 * E2

In [29]:
reactions = []
for reaction in [r for r in simple.reactions if r.category!=cn.REACTION_BOUNDARY]:
  if (reaction.reactants[0].molecule.name == cn.EMPTYSET) | \
      (reaction.products[0].molecule.name == cn.EMPTYSET):
   continue
  else:
    reactions.append(reaction)
# for reaction in reactions:
#   print(reaction)
z = m.analyze(reactions)

We have a Type I Error...
AA  and  Cn  have the same weight by
v8: _1DG -> Cn; k8 * _1DG
v12: Man -> Glu; k12 * Man
v9: _1DG -> AA; k9 * _1DG
v13: Glu -> _3DG; k13 * Glu
v5: _3DG -> Cn; k5 * _3DG
v6: _3DG -> FA; k6 * _3DG

However, reaction " v15: Cn -> AA + FA + MG; k15 * Cn " implies  AA < Cn
We cannot add the arc:  AA -> Cn

We have a Type I Error...
FA  and  Cn  have the same weight by
v8: _1DG -> Cn; k8 * _1DG
v12: Man -> Glu; k12 * Man
v9: _1DG -> AA; k9 * _1DG
v13: Glu -> _3DG; k13 * Glu
v5: _3DG -> Cn; k5 * _3DG
v6: _3DG -> FA; k6 * _3DG

However, reaction " v15: Cn -> AA + FA + MG; k15 * Cn " implies  FA < Cn
We cannot add the arc:  FA -> Cn

